<a href="https://colab.research.google.com/github/sushirito/Hg2/blob/main/2D_Diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyDOE
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.size'] = 24
plt.rcParams['axes.linewidth'] = 2
import torch
import torch.nn as nn
from pyDOE import *
import time
import scipy.special as sp
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
torch.manual_seed(31)
np.random.seed(31)

class MLP(nn.Module):
    def __init__(self, layers):
        super(MLP, self).__init__()
        self.activation = nn.Tanh()
        self.loss_function = nn.MSELoss(reduction='mean')
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        self.iter = 0
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            nn.init.zeros_(self.linears[i].bias.data)

    def forward(self, x):
        for i in range(len(self.linears)-1):
            x = self.activation(self.linears[i](x))
        x = self.linears[-1](x)
        return x

    def loss_bc_init(self, x, y):
        loss_u = self.loss_function(self.forward(x), y)
        return loss_u

    def loss_initernal(self, x_train):
        x_train.requires_grad = True
        p = self.forward(x_train)
        u_g = torch.autograd.grad(p, x_train, grad_outputs=torch.ones_like(p), create_graph=True)[0]
        u_x, u_y, u_t = u_g[:, [0]], u_g[:, [1]], u_g[:, [2]]
        u_xx = torch.autograd.grad(u_x, x_train, grad_outputs=torch.ones_like(u_x), create_graph=True)[0][:, [0]]
        u_yy = torch.autograd.grad(u_y, x_train, grad_outputs=torch.ones_like(u_y), create_graph=True)[0][:, [1]]
        pde = u_t - D * (u_xx + u_yy)
        loss_pde = pde.pow(2).mean()
        return loss_pde

    def loss(self, x, y, x_to_train_f):
        loss_u = self.loss_bc_init(x, y)
        loss_f = self.loss_initernal(x_to_train_f)
        return loss_u + loss_f

def nptoTensor(data):
    return torch.from_numpy(data).to(device).float()

def u_2d(x_2d):
    return np.sin(np.pi * x_2d[:, 0]) * np.sin(np.pi * x_2d[:, 1])

def trainingdata(Nx, Ny, Nt, Nf, Nu):
    x = np.linspace(0, 1, Nx)
    y = np.linspace(0, 1, Ny)
    t = np.linspace(0, 1, Nt)
    X, Y = np.meshgrid(x, y)
    x_2d = np.concatenate((X.flatten()[:, None], Y.flatten()[:, None]), axis=1)
    t_2d = np.zeros((Nx * Ny, 1))
    intial_condition = np.concatenate((x_2d, t_2d), axis=1)
    u_inital = u_2d(x_2d)[:, None]
    lower_x = np.concatenate((X[:, 0][:, None], Y[:, 0][:, None]), axis=1)
    upper_x = np.concatenate((X[:, -1][:, None], Y[:, 0][:, None]), axis=1)
    right_y = np.concatenate((X[0, :][:, None], Y[-1, :][:, None]), axis=1)
    left_y = np.concatenate((X[0, :][:, None], Y[0, :][:, None]), axis=1)
    x_bound = np.vstack([lower_x, upper_x, right_y, left_y])
    x_bound_ext = np.tile(x_bound, [Nt, 1])
    t_bound_ext = np.tile(t[:, None], [Nx * 4, 1])
    xt_bound_ext = np.concatenate((x_bound_ext, t_bound_ext), axis=1)
    u_bound_ext = np.zeros((Nx * 4 * Nt))[:, None]
    all_Init_bcs = np.vstack([intial_condition, xt_bound_ext])
    all_u_init_bcs = np.vstack([u_inital, u_bound_ext])
    ldx = np.random.permutation(all_Init_bcs.shape[0])
    all_Init_bcs = all_Init_bcs[ldx]
    all_u_init_bcs = all_u_init_bcs[ldx]
    f_train = lhs(3, Nf)
    f_train = np.vstack((f_train, all_Init_bcs))
    return all_Init_bcs, all_u_init_bcs, f_train

N_u = 500
N_x = 30
N_y = 30
N_t = 50
N_f = 10000
D = 0.1

# Generate training data
init_cond_train_np_array, u_init_cond_train_np_array, f_train_np_array = trainingdata(N_x, N_y, N_t, N_f, N_u)
init_cond_train = nptoTensor(init_cond_train_np_array)
u_init_cond_train = nptoTensor(u_init_cond_train_np_array)
f_train = nptoTensor(f_train_np_array)

# Model setup
neurons = 50
layers = np.array([3, neurons, neurons, neurons, neurons, neurons, neurons, neurons, neurons, neurons, neurons, neurons, neurons, neurons, 1])
mlp = MLP(layers).to(device)

# Training loop
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.0001)
A_hist = []
max_iter = 35000
tic = time.time()

for i in range(max_iter):
    loss = mlp.loss(init_cond_train, u_init_cond_train, f_train)
    A_hist.append([i, loss.detach().cpu().item()])
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if (i + 1) % 100 == 0:
        print(f"Epoch: {i+1}, MSE: {loss.detach().cpu().item():.6f}")

toc = time.time()
print(f'Total training time: {(toc - tic)/60:.2f} minutes')

# FDM Simulation
def run_2d_fdm_simulation():
    points = 20
    x = np.linspace(0, 1, points)
    y = np.linspace(0, 1, points)
    delta_x = x[1] - x[0]
    delta_y = y[1] - y[0]
    delta_t = 0.5 * (delta_x**2 * delta_y**2) / ((delta_x**2 + delta_y**2) * D)
    t_points = int(1.0/delta_t)
    t = np.linspace(0, 1, t_points)
    u = np.zeros((len(t), len(x), len(y)))

    # Initial condition
    X, Y = np.meshgrid(x[1:-1], y[1:-1])
    u[0,1:-1,1:-1] = np.sin(np.pi*X) * np.sin(np.pi*Y)

    # Time integration
    for n in range(len(t)-1):
        for i in range(1, len(x)-1):
            for j in range(1, len(y)-1):
                u[n+1,i,j] = u[n,i,j] + delta_t*D*(
                    (u[n,i+1,j] - 2*u[n,i,j] + u[n,i-1,j])/delta_x**2 +
                    (u[n,i,j+1] - 2*u[n,i,j] + u[n,i,j-1])/delta_y**2
                )
        # Boundary conditions
        u[n+1,[0,-1],:] = 0
        u[n+1,:,[0,-1]] = 0
    return x, y, t, u

x_fdm, y_fdm, t_fdm, u_fdm = run_2d_fdm_simulation()

def analytical_solution(x, y, t, D):
    return np.sin(np.pi*x) * np.sin(np.pi*y) * np.exp(-2*(np.pi**2)*D*t)

def eval_model(m):
    mlp.eval()
    with torch.no_grad():
        p = mlp(torch.tensor(m).float().to(device)).cpu().numpy()
    return p

def plot_3d_surfaces(x_fdm, y_fdm, t_fdm, u_fdm):
    times = [0, 0.25, 0.5, 0.75, 1]
    fig = plt.figure(figsize=(25, 15))

    for idx, t_val in enumerate(times):
        ax = fig.add_subplot(2, 3, idx+1, projection='3d')
        X, Y = np.meshgrid(x_fdm, y_fdm)
        m = np.hstack([X.reshape(-1,1), Y.reshape(-1,1), t_val*np.ones((400,1))])

        pinn = eval_model(m).reshape(20,20)
        analytic = analytical_solution(X, Y, t_val, D)
        fdm_idx = int(np.round(t_val * (len(t_fdm)-1)))

        ax.plot_surface(X, Y, pinn, cmap='rainbow', alpha=0.6, label='PINN')
        ax.plot_surface(X, Y, analytic, cmap='viridis', alpha=0.3, label='Analytical')
        ax.plot_wireframe(X, Y, u_fdm[fdm_idx], color='black', alpha=0.5, label='FDM')
        ax.set_title(f't = {t_val:.2f}')
        ax.set_zlim(0,1)
    plt.savefig('3D_Surface_Comparison.png', dpi=300)

def plot_error_snapshots(x_fdm, y_fdm, t_fdm, u_fdm):
    times = [0.25, 0.5, 0.75, 1]
    fig, axs = plt.subplots(2, 2, figsize=(20, 20))

    for idx, t_val in enumerate(times):
        ax = axs[idx//2, idx%2]
        X, Y = np.meshgrid(x_fdm, y_fdm)
        m = np.hstack([X.reshape(-1,1), Y.reshape(-1,1), t_val*np.ones((400,1))])

        pinn = eval_model(m).reshape(20,20)
        analytic = analytical_solution(X, Y, t_val, D)
        fdm_idx = int(np.round(t_val * (len(t_fdm)-1)))

        err = np.abs(pinn - analytic)
        im = ax.imshow(err.T, origin='lower', extent=[0,1,0,1], cmap='viridis')
        plt.colorbar(im, ax=ax)
        ax.set_title(f't = {t_val:.2f}')
    plt.savefig('Error_Snapshots.png', dpi=300)

def plot_convergence_metrics():
    fig, ax1 = plt.subplots(figsize=(12,6))
    ax1.semilogy(np.array(A_hist)[:,0], np.array(A_hist)[:,1], 'b-')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Loss (log scale)', color='b')
    plt.savefig('Convergence.png', dpi=300)

def plot_error_heatmaps(x_fdm, y_fdm, t_fdm, u_fdm):
    fig, axs = plt.subplots(1, 3, figsize=(30,8))
    t_vals = [0.25, 0.5, 1.0]

    for idx, t_val in enumerate(t_vals):
        X, Y = np.meshgrid(x_fdm, y_fdm)
        m = np.hstack([X.reshape(-1,1), Y.reshape(-1,1), t_val*np.ones((400,1))])

        pinn = eval_model(m).reshape(20,20)
        analytic = analytical_solution(X, Y, t_val, D)
        err = np.abs(pinn - analytic)

        im = axs[idx].imshow(err.T, norm=mpl.colors.LogNorm(vmin=1e-4, vmax=0.1),
                           origin='lower', extent=[0,1,0,1], cmap='viridis')
        plt.colorbar(im, ax=axs[idx])
        axs[idx].set_title(f't = {t_val:.2f}')
    plt.savefig('Error_Heatmaps.png', dpi=300)

def plot_temporal_evolution(x_fdm, y_fdm, t_fdm, u_fdm):
    points = [(0.25,0.25), (0.5,0.5), (0.75,0.75)]
    fig, axs = plt.subplots(3, 1, figsize=(10,15))

    for idx, (x_val, y_val) in enumerate(points):
        t_vals = np.linspace(0, 1, 50)
        pinn_t = []
        analytic_t = []
        fdm_t = []

        for ti in t_vals:
            m = np.array([[x_val, y_val, ti]])
            pinn_t.append(eval_model(m)[0])
            analytic_t.append(analytical_solution(x_val, y_val, ti, D))
            fdm_idx = int(np.round(ti * (len(t_fdm)-1)))
            fdm_idx_x = np.argmin(np.abs(x_fdm - x_val))
            fdm_idx_y = np.argmin(np.abs(y_fdm - y_val))
            fdm_t.append(u_fdm[fdm_idx, fdm_idx_x, fdm_idx_y])

        axs[idx].plot(t_vals, analytic_t, 'b-', label='Analytical')
        axs[idx].plot(t_vals, pinn_t, 'r--', label='PINN')
        axs[idx].plot(t_vals, fdm_t, 'g:', label='FDM')
        axs[idx].set_title(f'(x,y) = ({x_val:.2f},{y_val:.2f})')
        axs[idx].grid(True)
    plt.savefig('Temporal_Evolution.png', dpi=300)

# Generate all plots
plot_3d_surfaces(x_fdm, y_fdm, t_fdm, u_fdm)
plot_error_snapshots(x_fdm, y_fdm, t_fdm, u_fdm)
plot_convergence_metrics()
plot_error_heatmaps(x_fdm, y_fdm, t_fdm, u_fdm)
plot_temporal_evolution(x_fdm, y_fdm, t_fdm, u_fdm)

# Save final model
path = '2D-Diffusion_PDE.pth'
torch.save(mlp.state_dict(), path)

  Preparing metadata (setup.py) ... done
  Created wheel for pyDOE: filename=pyDOE-0.3.8-py3-none-any.whl size=18170 sha256=1d74e44f27ad17ca86c3dd3bc1325e506c16ebc5d2dd471a56ca146fe19e7b79
  Stored in directory: /root/.cache/pip/wheels/84/20/8c/8bd43ba42b0b6d39ace1219d6da1576e0dac81b12265c4762e
Successfully built pyDOE
cuda


/usr/local/lib/python3.11/dist-packages/torch/autograd/graph.py:825: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at ../aten/src/ATen/cuda/CublasHandlePool.cpp:135.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch: 100, MSE: 0.026761
Epoch: 200, MSE: 0.024696
Epoch: 300, MSE: 0.022623
Epoch: 400, MSE: 0.018416
Epoch: 500, MSE: 0.013206
Epoch: 600, MSE: 0.009391
Epoch: 700, MSE: 0.006929
Epoch: 800, MSE: 0.005489
Epoch: 900, MSE: 0.004630
Epoch: 1000, MSE: 0.004010
Epoch: 1100, MSE: 0.003545
Epoch: 1200, MSE: 0.003325
Epoch: 1300, MSE: 0.002806
Epoch: 1400, MSE: 0.002499
Epoch: 1500, MSE: 0.002211
Epoch: 1600, MSE: 0.001965
Epoch: 1700, MSE: 0.001998
Epoch: 1800, MSE: 0.001566
Epoch: 1900, MSE: 0.001461
Epoch: 2000, MSE: 0.001291
Epoch: 2100, MSE: 0.001577
Epoch: 2200, MSE: 0.001097
Epoch: 2300, MSE: 0.001012
Epoch: 2400, MSE: 0.000951
Epoch: 2500, MSE: 0.000879
Epoch: 2600, MSE: 0.001042
Epoch: 2700, MSE: 0.000761
Epoch: 2800, MSE: 0.000700
Epoch: 2900, MSE: 0.000679
Epoch: 3000, MSE: 0.000598
Epoch: 3100, MSE: 0.003129
Epoch: 3200, MSE: 0.000509
Epoch: 3300, MSE: 0.000462
Epoch: 3400, MSE: 0.000501
Epoch: 3500, MSE: 0.000392
Epoch: 3600, MSE: 0.000354
Epoch: 3700, MSE: 0.000339
Epoch: 380

In [ ]:
def analytical_solution(x, y, t, D):
    return np.sin(np.pi*x) * np.sin(np.pi*y) * np.exp(-2*D*(np.pi**2)*t)

def plot_convergence():
    plt.figure(figsize=(10,6))
    plt.semilogy(np.array(A_hist)[:,0], np.array(A_hist)[:,1], 'b-', lw=2)
    plt.xlabel('Epochs', fontsize=14)
    plt.ylabel('Loss (log scale)', fontsize=14)
    plt.title('Training Convergence', fontsize=16)
    plt.grid(True, alpha=0.3)
    plt.savefig('convergence.png', dpi=300, bbox_inches='tight')

def plot_contour_comparison(t_val=0.5):
    X, Y = np.meshgrid(np.linspace(0,1,20), np.linspace(0,1,20))
    m = np.hstack([X.reshape(-1,1), Y.reshape(-1,1), t_val*np.ones((400,1))])

    # Get preds
    pinn_pred = eval_model(m).reshape(20,20)
    analytic = analytical_solution(X, Y, t_val, D)
    fdm_idx = int(np.round(t_val * (len(t_fdm)-1)))

    fig, axes = plt.subplots(1, 3, figsize=(18,5))
    titles = ['PINN', 'FDM', 'Analytical']
    data = [pinn_pred, u_fdm[fdm_idx], analytic]

    for ax, d, title in zip(axes, data, titles):
        cnt = ax.contourf(X, Y, d, levels=50, cmap='viridis')
        ax.set_title(title, fontsize=12)
        ax.set_xlabel('x')
        ax.set_ylabel('y')
        fig.colorbar(cnt, ax=ax)

    plt.savefig(f'contour_comparison_t_{t_val:.2f}.png', dpi=300)

def plot_model(fixed_var='x', fixed_val=0.5):
    fig = plt.figure(figsize=(18,6))
    var_names = {'x': (1,2), 'y': (0,2)}  # (varying dims, fixed dim)

    t = np.linspace(0,1,50)
    var = np.linspace(0,1,50)
    T, VAR = np.meshgrid(t, var)

    if fixed_var == 'x':
        coords = np.c_[fixed_val*np.ones_like(VAR.ravel()), VAR.ravel(), T.ravel()]
    else:
        coords = np.c_[VAR.ravel(), fixed_val*np.ones_like(VAR.ravel()), T.ravel()]

    pinn = eval_model(coords).reshape(50,50)
    analytic = analytical_solution(coords[:,0], coords[:,1], coords[:,2], D).reshape(50,50)

    fdm_vals = np.array([[np.interp(ti, t_fdm, u_fdm[:,int(fixed_val*19),int(varj*19)])
                        for ti in t] for varj in var])

    ax = fig.add_subplot(131, projection='3d')
    ax.plot_surface(T, VAR, pinn, cmap='viridis')
    ax.set_title('PINN Solution')

    ax = fig.add_subplot(132, projection='3d')
    ax.plot_surface(T, VAR, fdm_vals, cmap='plasma')
    ax.set_title('FDM Solution')

    ax = fig.add_subplot(133, projection='3d')
    ax.plot_surface(T, VAR, analytic, cmap='magma')
    ax.set_title('Analytical Solution')

    plt.savefig(f'3d_comparison_fixed_{fixed_var}_{fixed_val:.2f}.png', dpi=300)

def plot_3d_surfaces():
    X, Y = np.meshgrid(np.linspace(0,1,20), np.linspace(0,1,20))

    fig = plt.figure(figsize=(18,12))
    times = [0, 0.25, 0.5, 0.75, 1]

    for idx, t_val in enumerate(times):
        ax = fig.add_subplot(2, 3, idx+1, projection='3d')
        m = np.hstack([X.ravel()[:,None], Y.ravel()[:,None], t_val*np.ones((400,1))])

        pinn = eval_model(m).reshape(20,20)
        analytic = analytical_solution(X, Y, t_val, D)
        fdm_idx = int(t_val * (len(t_fdm)-1))

        surf = ax.plot_surface(X, Y, pinn, cmap='viridis', alpha=0.8)
        ax.contour(X, Y, analytic, zdir='z', offset=pinn.min(), cmap='hot')
        ax.scatter(X, Y, u_fdm[fdm_idx], c='r', s=10)

        ax.set_title(f't = {t_val:.2f}')
        ax.set_zlim(0,1)

    plt.savefig('3d_surface_comparisons.png', dpi=300)

def plot_error_snapshots():
    fig, axes = plt.subplots(2, 2, figsize=(16,12))
    times = [0.25, 0.5, 0.75, 1.0]

    for idx, t_val in enumerate(times):
        ax = axes[idx//2, idx%2]
        X, Y = np.meshgrid(np.linspace(0,1,20), np.linspace(0,1,20))
        m = np.hstack([X.ravel()[:,None], Y.ravel()[:,None], t_val*np.ones((400,1))])

        pinn = eval_model(m).reshape(20,20)
        analytic = analytical_solution(X, Y, t_val, D)
        err = np.abs(pinn - analytic)

        cnt = ax.contourf(X, Y, err, levels=50, cmap='hot', norm=mpl.colors.LogNorm())
        ax.set_title(f't = {t_val:.2f}')
        fig.colorbar(cnt, ax=ax)

    plt.savefig('error_snapshots.png', dpi=300)

def plot_temporal_evolution():
    points = [(0.25,0.25), (0.5,0.5), (0.75,0.75)]
    fig, axes = plt.subplots(3, 1, figsize=(10,12))

    for idx, (x_val, y_val) in enumerate(points):
        t = np.linspace(0,1,50)
        pinn = []
        analytic = []
        fdm = []

        for ti in t:
            m = np.array([[x_val, y_val, ti]])
            pinn.append(eval_model(m)[0])
            analytic.append(analytical_solution(x_val, y_val, ti, D))
            tidx = int(ti * (len(t_fdm)-1))
            fdm.append(u_fdm[tidx, int(x_val*19), int(y_val*19)])

        axes[idx].plot(t, analytic, 'b-', label='Analytical')
        axes[idx].plot(t, pinn, 'r--', label='PINN')
        axes[idx].plot(t, fdm, 'g:', label='FDM')
        axes[idx].set_title(f'Position ({x_val:.2f}, {y_val:.2f})')
        axes[idx].grid(True)
        if idx == 0:
            axes[idx].legend()

    plt.savefig('temporal_evolution.png', dpi=300)

# Gen all plots
plot_convergence()
plot_contour_comparison()
plot_model(fixed_var='x', fixed_val=0.5)
plot_model(fixed_var='y', fixed_val=0.5)
plot_3d_surfaces()
plot_error_snapshots()
plot_temporal_evolution()

#More Visuals

In [ ]:
def plot_unified_3d_comparison():
    times = [0, 0.25, 0.5, 0.75, 1]
    fig = plt.figure(figsize=(20, 15))

    for idx, t_val in enumerate(times):
        ax = fig.add_subplot(2, 3, idx+1, projection='3d')
        X, Y = np.meshgrid(np.linspace(0,1,20), np.linspace(0,1,20))
        m = np.hstack([X.ravel()[:,None], Y.ravel()[:,None], t_val*np.ones((400,1))])

        pinn = eval_model(m).reshape(20,20)
        analytic = analytical_solution(X, Y, t_val, D)
        fdm_idx = int(t_val * (len(t_fdm)-1))

        surf = ax.plot_surface(X, Y, pinn, cmap='viridis', alpha=0.8, label='PINN')
        ax.contour(X, Y, analytic, zdir='z', offset=0, cmap='hot', label='Analytical')
        ax.scatter(X, Y, u_fdm[fdm_idx], c='r', s=10, label='FDM')

        ax.set_xlabel('x-coordinate')
        ax.set_ylabel('y-coordinate')
        ax.set_zlabel('Solution u(x,y,t)')
        ax.set_title(f'Time t = {t_val:.2f}')
        ax.set_zlim(0, 1)

    plt.suptitle('Evolution of 2D Diffusion: PINN vs Analytical vs FDM', fontsize=16)
    plt.tight_layout()


In [ ]:
def plot_error_composite():
    times = [0.25, 0.5, 0.75, 1]
    fig, axes = plt.subplots(2, 2, figsize=(15, 15))

    for idx, t_val in enumerate(times):
        ax = axes[idx//2, idx%2]
        X, Y = np.meshgrid(np.linspace(0,1,20), np.linspace(0,1,20))
        m = np.hstack([X.ravel()[:,None], Y.ravel()[:,None], t_val*np.ones((400,1))])

        pinn = eval_model(m).reshape(20,20)
        analytic = analytical_solution(X, Y, t_val, D)
        error = np.abs(pinn - analytic)

        im = ax.imshow(error.T, origin='lower', extent=[0,1,0,1],
                      norm=mpl.colors.LogNorm(vmin=1e-4, vmax=1e-1),
                      cmap='viridis')
        ax.set_xlabel('x-coordinate')
        ax.set_ylabel('y-coordinate')
        ax.set_title(f'Absolute Error at t = {t_val:.2f}')
        plt.colorbar(im, ax=ax, label='|PINN - Analytical|')

    plt.suptitle('Spatial Distribution of PINN Solution Error', fontsize=16)
    plt.tight_layout()


In [ ]:
def plot_enhanced_convergence():
    fig, ax = plt.subplots(figsize=(10, 6))
    epochs = np.array(A_hist)[:,0]
    loss = np.array(A_hist)[:,1]

    ax.semilogy(epochs, loss, 'b-', linewidth=2, label='Training Loss')
    ax.grid(True, which="both", ls="-", alpha=0.2)
    ax.set_xlabel('Training Epochs')
    ax.set_ylabel('Mean Squared Error (log scale)')
    ax.set_title('PINN Training Convergence History')

    ax.annotate(f'Final Loss: {loss[-1]:.2e}',
                xy=(epochs[-1], loss[-1]),
                xytext=(-50, 20), textcoords='offset points',
                bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.5))
    plt.tight_layout()


In [ ]:
def plot_cross_sections():
    fig = plt.figure(figsize=(20, 8))
    x_val, y_val = 0.5, 0.5
    t = np.linspace(0, 1, 50)
    var = np.linspace(0, 1, 50)
    T, VAR = np.meshgrid(t, var)

    ax1 = fig.add_subplot(121, projection='3d')
    coords_x = np.c_[x_val*np.ones_like(VAR.ravel()),
                    VAR.ravel(), T.ravel()]
    pinn_x = eval_model(coords_x).reshape(50, 50)
    surf1 = ax1.plot_surface(T, VAR, pinn_x, cmap='viridis')
    ax1.set_xlabel('Time (t)')
    ax1.set_ylabel('y-coordinate')
    ax1.set_zlabel('u(x=0.5,y,t)')
    ax1.set_title('Solution Evolution at x = 0.5')
    plt.colorbar(surf1, ax=ax1, label='Solution Value')

    ax2 = fig.add_subplot(122, projection='3d')
    coords_y = np.c_[VAR.ravel(),
                    y_val*np.ones_like(VAR.ravel()), T.ravel()]
    pinn_y = eval_model(coords_y).reshape(50, 50)
    surf2 = ax2.plot_surface(T, VAR, pinn_y, cmap='viridis')
    ax2.set_xlabel('Time (t)')
    ax2.set_ylabel('x-coordinate')
    ax2.set_zlabel('u(x,y=0.5,t)')
    ax2.set_title('Solution Evolution at y = 0.5')
    plt.colorbar(surf2, ax=ax2, label='Solution Value')

    plt.suptitle('Cross-Sectional Analysis of PINN Solution', fontsize=16)
    plt.tight_layout()


In [ ]:
def plot_temporal_tracking():
    points = [(0.25,0.25), (0.5,0.5), (0.75,0.75)]
    fig, axes = plt.subplots(3, 1, figsize=(12, 15))
    t = np.linspace(0, 1, 50)

    for idx, (x_val, y_val) in enumerate(points):
        m = np.array([[x_val, y_val, ti] for ti in t])
        pinn = eval_model(m).reshape(-1)
        analytic = analytical_solution(x_val, y_val, t, D)

        axes[idx].plot(t, analytic, 'b-', label='Analytical', linewidth=2)
        axes[idx].plot(t, pinn, 'r--', label='PINN', linewidth=2)
        axes[idx].grid(True, alpha=0.3)
        axes[idx].set_xlabel('Time (t)')
        axes[idx].set_ylabel('u(x,y,t)')
        axes[idx].set_title(f'Solution at (x,y) = ({x_val:.2f},{y_val:.2f})')
        axes[idx].legend()

    plt.suptitle('Temporal Evolution at Selected Points', fontsize=16)
    plt.tight_layout()


In [ ]:
def plot_enhanced_contours():
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    t_val = 0.5
    X, Y = np.meshgrid(np.linspace(0,1,20), np.linspace(0,1,20))
    m = np.hstack([X.ravel()[:,None], Y.ravel()[:,None],
                   t_val*np.ones((400,1))])

    pinn = eval_model(m).reshape(20,20)
    analytic = analytical_solution(X, Y, t_val, D)
    fdm_idx = int(t_val * (len(t_fdm)-1))

    titles = ['PINN', 'Analytical', 'FDM']
    solutions = [pinn, analytic, u_fdm[fdm_idx]]

    for ax, sol, title in zip(axes, solutions, titles):
        cnt = ax.contourf(X, Y, sol, levels=50, cmap='viridis')
        ax.set_xlabel('x-coordinate')
        ax.set_ylabel('y-coordinate')
        ax.set_title(f'{title} Solution at t = {t_val}')
        plt.colorbar(cnt, ax=ax, label='u(x,y,t)')

    plt.suptitle('Contour Comparison of Solutions', fontsize=16)
    plt.tight_layout()


In [ ]:
def plot_boundary_verification():
    t_vals = [0, 0.25, 0.5, 0.75, 1.0]
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.ravel()

    for idx, t_val in enumerate(t_vals):
        x_bound = np.linspace(0, 1, 100)
        y_bound = np.zeros_like(x_bound)
        m_bottom = np.hstack([x_bound[:,None], y_bound[:,None],
                            t_val*np.ones_like(x_bound)[:,None]])
        m_top = np.hstack([x_bound[:,None], np.ones_like(x_bound)[:,None],
                          t_val*np.ones_like(x_bound)[:,None]])

        u_bottom = eval_model(m_bottom)
        u_top = eval_model(m_top)

        axes[idx].plot(x_bound, u_bottom, 'b-', label='Bottom BC')
        axes[idx].plot(x_bound, u_top, 'r--', label='Top BC')
        axes[idx].grid(True, alpha=0.3)
        axes[idx].set_xlabel('x-coordinate')
        axes[idx].set_ylabel('u(x,y,t)')
        axes[idx].set_title(f't = {t_val:.2f}')
        if idx == 0:
            axes[idx].legend()

    plt.suptitle('Boundary Condition Verification', fontsize=16)
    plt.tight_layout()


In [ ]:
plot_unified_3d_comparison()
plot_error_composite()
plot_enhanced_convergence()
plot_cross_sections()
plot_temporal_tracking()
plot_enhanced_contours()
plot_boundary_verification()